#### This code is to perform error analysis on a random number of images selected from different datasets

In [1]:
import base64
import httpx
import anthropic
import json
import random
import glob as glob
from collections import Counter

from openai import OpenAI 
import os
import pandas as pd
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
import time
from Important_script_1 import *

In [2]:
DATASET_NAME = "DermaMNIST-4A"
NUMBER_OF_IMAGES = 100

In [3]:
dataset_class_name_dict = {
        "DermaMNIST-2": {
        '0': 'actinic keratoses and intraepithelial carcinoma',
        '1': 'basal cell carcinoma',
        # '2': 'benign keratosis-like lesions',
        # '3': 'dermatofibroma',
        # '4': 'melanoma',
        # '5': 'melanocytic nevi',
        # '6': 'vascular lesions'
    },

    "DermaMNIST-4A": {
        # '0': 'actinic keratoses and intraepithelial carcinoma',
        # '1': 'basal cell carcinoma',
        # '2': 'benign keratosis-like lesions',
        '3': 'dermatofibroma',
        '4': 'melanoma',
        '5': 'melanocytic nevi',
        '6': 'vascular lesions'
    },
        "DermaMNIST-4": {
        '0': 'actinic keratoses and intraepithelial carcinoma',
        '1': 'basal cell carcinoma',
        '2': 'benign keratosis-like lesions',
        '3': 'dermatofibroma',
        # '4': 'melanoma',
        # '5': 'melanocytic nevi',
        # '6': 'vascular lesions'
    },

    "DermaMNIST": {
        '0': 'actinic keratoses and intraepithelial carcinoma',
        '1': 'basal cell carcinoma',
        '2': 'benign keratosis-like lesions',
        '3': 'dermatofibroma',
        '4': 'melanoma',
        '5': 'melanocytic nevi',
        '6': 'vascular lesions'
    },

    "OctMNIST-2A": {
        # "0": "choroidal neovascularization",
        # "1": "diabetic macular edema",
        "2": "drusen",
        "3": "normal",
    },

    "OctMNIST-2": {
        "0": "choroidal neovascularization",
        "1": "diabetic macular edema",
        # "2": "drusen",
        # "3": "normal",
    },

    "OctMNIST": {
        "0": "choroidal neovascularization",
        "1": "diabetic macular edema",
        "2": "drusen",
        "3": "normal",
    },

    'PneumoniaMNIST': {
        "0": "normal", 
        "1": "pneumonia"
        }
}

In [4]:
source_directory = f"../Datasets/{DATASET_NAME}/test"  # Change this to your test directory path

# Get randomly selected files
selected = select_random_files(source_dir=source_directory, n_files=100, random_state=42)
image_paths = []

# Print results
for folder, files in selected.items():
    for file in files:
        # print(f"  - {file}")
        image_paths.append(f"{source_directory}/{folder}/{file}")

import random

# Set random seed for reproducibility (optional)
random.seed(42)

my_list = image_paths

image_paths = random.sample(my_list, NUMBER_OF_IMAGES)

In [5]:
dataset_classes_of_interest =  dataset_class_name_dict[DATASET_NAME]
classes_of_interest = list(dataset_classes_of_interest.values())

#### Playground

In [6]:
pred_classes_openai = generate_medical_diagnosis_parallel(image_paths, classes_of_interest, "openai", max_workers=2, batch_size=10)

Total Progress:   0%|          | 0/100 [00:00<?, ?image/s]

In [ ]:
# pred_classes_claude = generate_medical_diagnosis_parallel(image_paths, classes_of_interest, "claude", max_workers=2, batch_size=10)

Total Progress:   0%|          | 0/100 [00:00<?, ?image/s]

In [7]:
def collect_class_predictions(class_predictions):
    list_of_predicted_classes = []
    list_of_correct_classes = []
    for image_name, prediction in class_predictions.items():
    # print(f"Image: {image_name}")
    # Get the predicted class
        predicted_class = prediction['predictedClass']
        list_of_predicted_classes.append(predicted_class)

    # Get the correct class
        file_path = prediction['file_path']
        correct_class = file_path.split('/')[-2]
        list_of_correct_classes.append(dataset_class_name_dict[DATASET_NAME][correct_class])

    return list_of_predicted_classes, list_of_correct_classes

In [8]:
list_of_predicted_classes_openai, list_of_correct_classes_openai = collect_class_predictions(pred_classes_openai)

In [ ]:
# list_of_predicted_classes_claude, list_of_correct_classes_claude = collect_class_predictions(pred_classes_claude)

In [9]:
count_matches(list_of_correct_classes_openai, list_of_predicted_classes_openai)

40

In [ ]:
# count_matches(list_of_correct_classes_claude, list_of_predicted_classes_claude)

48

In [10]:
pd.DataFrame(pred_classes_openai).T.to_csv("openai_gpt-4o-mini_predictions_zero_shot_direct_dermamnist_reduced_class_4A_V3.csv")
# pd.DataFrame(pred_classes_claude).T.to_csv("claude_predictions_zero_shot_direct_dermamnist_reduced_class_4A_V3.csv")

In [12]:
image_paths_list = image_paths[18:19]

In [10]:
unbiased_features = analyze_images_unbiased_parallel(image_paths_list, classes_of_interest, model_type="openai", max_retries=3, max_workers=2)

Processing images: 100%|██████████| 1/1 [00:11<00:00, 11.54s/it]


In [11]:
unbiased_features

{'../Datasets/DermaMNIST/test/1/image_293.png': ["Error: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}"]}

In [61]:
base_features = extract_base_features(unbiased_features)

In [65]:
c0 = generate_single_class_description_test_test(image_paths_list[0], classes_of_interest, classes_of_interest[0], base_features=base_features)
c1 = generate_single_class_description_test_test(image_paths_list[0], classes_of_interest, classes_of_interest[1], base_features=base_features)
c2 = generate_single_class_description_test_test(image_paths_list[0], classes_of_interest, classes_of_interest[2], base_features=base_features)
c3 = generate_single_class_description_test_test(image_paths_list[0], classes_of_interest, classes_of_interest[3], base_features=base_features)

In [66]:
find_closest_description(unbiased_features, [c0, c1, c2, c3], "openai")

(3,
 80,
 ["Candidate 3 describes a 'smooth surface', which matches the reference's 'smooth surface'.",
  "Candidate 3 mentions 'homogeneous texture', which is semantically similar to the reference's 'uniform texture'.",
  "Candidate 3 uses 'well-defined boundary', which is very similar to the reference's 'defined boundaries'.",
  "The overall pattern in Candidate 3 is more consistent with the reference due to the use of terms like 'uniform' and 'consistent', which align with the reference's 'layered structure' and 'central thickening'.",
  "Candidates 0, 1, and 2 describe features that are opposite or unrelated to the reference, such as 'irregular structure' and 'blurred boundary'."])

In [60]:
image_paths_list[0]

'Datasets/OctMNIST/test/1/700.png'

#### Playground 2

In [67]:
nc0 = generate_single_class_description(image_paths_list[0], classes_of_interest, classes_of_interest[0])
nc1 = generate_single_class_description(image_paths_list[0], classes_of_interest, classes_of_interest[1])
nc2 = generate_single_class_description(image_paths_list[0], classes_of_interest, classes_of_interest[2])
nc3 = generate_single_class_description(image_paths_list[0], classes_of_interest, classes_of_interest[3])

In [71]:
bs_features = extract_base_features(list(set(nc0 + nc1 + nc2 + nc3)))

In [72]:
image_feat_unbiased_descriptions = generate_unbiased_description_test_test(image_paths_list[0], classes_of_interest, list(set(bs_features)))

In [73]:
find_closest_description(image_feat_unbiased_descriptions, [nc0, nc1, nc2, nc3])

(3,
 75,
 ["Both describe smoothness in the structure ('smooth surface' vs 'smooth retinal layers')",
  "Both mention uniformity ('uniform pattern' vs 'uniform texture')",
  "Candidate 3 lacks a direct mention of 'clear boundary', but 'absence of subretinal fluid' suggests a clear delineation",
  "Overall, Candidate 3's features align more closely with the reference's emphasis on smoothness and uniformity"])

In [45]:
image_paths_list

['Datasets/OctMNIST/test/3/822.png']

In [47]:
nc1 = generate_single_class_description_test_test(image_paths_list[0], classes_of_interest, classes_of_interest[0], bs_features)
nc2 = generate_single_class_description_test_test(image_paths_list[0], classes_of_interest, classes_of_interest[1], bs_features)
nc3 = generate_single_class_description_test_test(image_paths_list[0], classes_of_interest, classes_of_interest[2], bs_features)
nc4 = generate_single_class_description_test_test(image_paths_list[0], classes_of_interest, classes_of_interest[3], bs_features)

In [48]:
find_closest_description(unbiased_descriptions, [nc0, nc1, nc2, nc3])

(3,
 60,
 ["Both sets describe 'localized thickening', which is similar to 'slight thickening' in the reference.",
  "The term 'irregular structure' in the candidate set contrasts with 'curved shape' and 'layered structure' in the reference, but both describe structural features.",
  "The candidate's 'discrete elevation' is somewhat similar to 'smooth elevation', though not identical.",
  "The presence of 'subretinal deposit' in the candidate set is opposite to 'absent deposit' in the reference, indicating a difference.",
  "The candidate's 'heterogeneous reflectivity' contrasts with 'uniform texture', but both address texture and reflectivity."])

In [35]:
unbiased_descriptions

['layered structure',
 'uniform pattern',
 'smooth surface',
 'clear boundary',
 'consistent thickness']

In [23]:
nc1

['irregular structure',
 'localized thickening',
 'heterogeneous reflectivity',
 'subretinal fluid accumulation',
 'disrupted retinal layers']

In [24]:
image_paths_list

['Datasets/OctMNIST/test/1/700.png']

In [18]:
find_closest_description(unbiased_descriptions, [nc0, nc1, nc2, nc3])

(3,
 75,
 ["Both describe 'smooth surface' with 'smooth retinal surface'",
  "Candidate 3 mentions 'uniform retinal layers', similar to 'uniform pattern'",
  "The 'consistent reflectivity across layers' in Candidate 3 aligns with 'variable reflectivity' in terms of discussing reflectivity, though not identical",
  "Absence of fluid accumulation in Candidate 3 suggests a 'clear boundary', as fluid presence often disrupts boundaries",
  "Overall, Candidate 3 describes a more organized and consistent retinal structure, similar to the reference's 'layered structure' and 'clear boundary'"])

In [20]:
nc1

['retinal thickening', 'intraretinal fluid accumulation']

In [46]:
image_paths_list[0]

'Datasets/OctMNIST/test/3/822.png'

In [17]:
biased_description = analyze_images_and_classes_parallel(image_paths[:1], classes_of_interest, model_type="openai")

Processing images: 100%|██████████| 1/1 [00:05<00:00,  5.51s/it]


In [7]:
r1 = generate_single_class_description_test_test(image_paths[0], classes_of_interest, classes_of_interest[0])

Processing images: 100%|██████████| 1/1 [00:06<00:00,  6.05s/it]


In [16]:
new_rs

{'Datasets/OctMNIST/test/2/797.png': {'choroidal neovascularization': ['irregular subretinal hyperreflective material',
   'disruption of retinal layers',
   'presence of subretinal fluid'],
  'diabetic macular edema': ['retinal thickening',
   'intraretinal fluid accumulation',
   'cystoid spaces'],
  'drusen': ['small, round, yellowish deposits',
   'discrete, well-defined borders',
   'scattered distribution beneath the retinal pigment epithelium'],
  'normal': ['uniform retinal layers',
   'absence of subretinal fluid',
   'smooth retinal contour',
   'lack of hyperreflective spots']}}

Processing images: 100%|██████████| 1/1 [00:01<00:00,  1.45s/it]


In [32]:
rs

{'Datasets/OctMNIST/test/0/857.png': ['layered structure',
  'central thickening',
  'irregular surface',
  'variations in reflectivity',
  'defined boundaries']}

In [8]:
rs = analyze_images_unbiased_parallel(image_paths[:1], classes_of_interest, model_type="openai", max_retries=3, max_workers=2)

Processing images: 100%|██████████| 1/1 [00:03<00:00,  3.26s/it]


In [33]:
extract_base_features(['layered structure',
  'central thickening',
  'irregular surface',
  'variations in reflectivity',
  'defined boundaries'])

['structure', 'thickening', 'surface', 'reflectivity', 'boundary']

In [10]:
rs = analyze_images_unbiased_parallel(image_paths[:1], classes_of_interest, model_type="openai", max_retries=3, max_workers=2)

Processing images: 100%|██████████| 1/1 [00:01<00:00,  1.18s/it]


In [11]:
rs

{'Datasets/OctMNIST/test/2/797.png': ['layered structure',
  'irregular undulations',
  'variations in reflectivity',
  'smooth surface areas',
  'distinct boundaries between layers']}

In [12]:
rs = analyze_images_unbiased_parallel(image_paths[:1], classes_of_interest, model_type="openai", max_retries=3, max_workers=2)

Processing images: 100%|██████████| 1/1 [00:02<00:00,  2.01s/it]


In [13]:
rs

{'Datasets/OctMNIST/test/2/797.png': ['layered structure',
  'wavy pattern',
  'variations in brightness',
  'smooth surface',
  'distinct boundaries']}

In [14]:
rs = analyze_images_unbiased_parallel(image_paths[:1], classes_of_interest, model_type="openai", max_retries=3, max_workers=2)
rs

Processing images: 100%|██████████| 1/1 [00:01<00:00,  1.66s/it]


{'Datasets/OctMNIST/test/2/797.png': ['layered structure',
  'wavy pattern',
  'variations in brightness',
  'smooth surface',
  'distinct boundaries']}

In [7]:
rs = analyze_images_and_classes_parallel(image_paths[:5], classes_of_interest, model_type="openai", max_retries=3, max_workers=2)

Processing images: 100%|██████████| 5/5 [00:25<00:00,  5.10s/it]


In [29]:
new_rs = analyze_images_and_classes_parallel(image_paths[:1], classes_of_interest, model_type="openai")

Processing images: 100%|██████████| 1/1 [00:08<00:00,  8.59s/it]


In [30]:
new_rs

{'Datasets/OctMNIST/test/2/797.png': {'choroidal neovascularization': ['irregular subretinal hyperreflective material',
   'disruption of retinal layers',
   'subretinal fluid accumulation'],
  'diabetic macular edema': ['retinal thickening',
   'cystoid spaces within the retina',
   'increased reflectivity in the macular region'],
  'drusen': ['small, round, hyperreflective deposits',
   "located between the retinal pigment epithelium and Bruch's membrane",
   'irregular elevation of the retinal pigment epithelium'],
  'normal': ['uniform retinal layers',
   'absence of subretinal fluid',
   'smooth retinal contour',
   'lack of hyperreflective spots']}}

In [7]:
rs = generate_single_class_description(image_paths[0], classes_of_interest, classes_of_interest[0], model_type="openai", image_media_type='image/png', max_retries=3)

In [8]:
rs

['subretinal hyperreflective material',
 'irregular retinal pigment epithelium elevation',
 'disruption of retinal layers']

In [9]:
rs = generate_single_class_description(image_paths[0], classes_of_interest, classes_of_interest[1], model_type="openai", image_media_type='image/png', max_retries=3)

In [10]:
rs

['retinal thickening', 'intraretinal cystoid spaces']

In [ ]:
rs = generate_single_class_description(image_paths[0], classes_of_interest, classes_of_interest[1], model_type="openai", image_media_type='image/png', max_retries=3)


#### Do the standard LLM class prediction exercise

In [7]:
pred_classes_openai = generate_medical_diagnosis_parallel(image_paths, classes_of_interest, "openai", max_workers=2)

Total Progress:   0%|          | 0/12 [00:00<?, ?image/s]

In [8]:
pred_classes_openai

{'image_11': {'file_path': '../Datasets/DermaMNIST/test/4/image_1190.png',
  'error': "Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}"},
 'image_12': {'file_path': '../Datasets/DermaMNIST/test/0/image_1127.png',
  'error': "Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}"},
 'image_1': {'file_path': '../Datasets/DermaMNIST/test/5/image_1985.png',
  'error': "Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing det

In [ ]:
# pred_classes_claude = generate_medical_diagnosis_parallel(image_paths, classes_of_interest, "claude", max_workers=2)

In [20]:
# pred_claude = [item['predictedClass'] for item in list(pred_classes_claude.values())]
# ground_claude = [dataset_classes_of_interest[item['file_path'].split("/")[-2]] for item in list(pred_classes_claude.values())]

In [32]:
pred_openai = [item['predictedClass'] for item in list(pred_classes_openai.values())]
ground_openai = [dataset_classes_of_interest[item['file_path'].split("/")[-2]] for item in list(pred_classes_openai.values())]

In [33]:
# print(f"Claude Accuracy: {count_matches(ground_claude, pred_claude)}")
print(f"OpenAI Accuracy: {count_matches(ground_openai, pred_openai)}")

OpenAI Accuracy: 14


#### Let us now use the feature descriptions

In [34]:
openai_feat_pred = generate_feature_descriptions_batch(image_paths, classes_of_interest, "openai", max_workers=2)


Processing 30 images with 2 workers


Processing Images:   0%|          | 0/30 [00:00<?, ?it/s]


Raw API response for Datasets/OctMNIST/test/2/797.png:

Raw API response for Datasets/OctMNIST/test/0/117.png:

Raw API response for Datasets/OctMNIST/test/3/948.png:

Attempt 1 failed for Datasets/OctMNIST/test/3/948.png:
Error: the JSON object must be str, bytes or bytearray, not NoneType

Raw API response for Datasets/OctMNIST/test/0/548.png:

Raw API response for Datasets/OctMNIST/test/1/244.png:

Attempt 1 failed for Datasets/OctMNIST/test/1/244.png:
Error: the JSON object must be str, bytes or bytearray, not NoneType

Raw API response for Datasets/OctMNIST/test/3/948.png:

Raw API response for Datasets/OctMNIST/test/1/244.png:

Raw API response for Datasets/OctMNIST/test/1/57.png:

Raw API response for Datasets/OctMNIST/test/0/833.png:

Attempt 1 failed for Datasets/OctMNIST/test/0/833.png:
Error: the JSON object must be str, bytes or bytearray, not NoneType

Raw API response for Datasets/OctMNIST/test/0/302.png:

Raw API response for Datasets/OctMNIST/test/0/833.png:

Raw API r

In [ ]:
# claude_feat_pred = generate_feature_descriptions_batch(image_paths, classes_of_interest, "claude", max_workers=2)

In [25]:
# claude_result_list, claude_item_list = get_class_descriptions_and_item_details(list(claude_feat_pred.values()))

In [40]:
def get_class_descriptions_and_item_details(result_list, number_of_classes):    
    second_result_list = []
    item_name_list = []
    for item in result_list:

        sec_res = {}
        for i in range(number_of_classes):
            sec_res[f'Class {i}'] = item[f'Class {i}']

        img_desc = {
            'Feature Description': item['Feature Description'],
            'Image Path': item['Image Path']
        }

        second_result_list.append(sec_res)
        item_name_list.append(img_desc)

    return second_result_list, item_name_list

In [38]:
openai_result_list, openai_item_list = get_class_descriptions_and_item_details(list(openai_feat_pred.values()))

In [27]:
# correct_class_paths_claude_output, claude_pred_class_paths_claude_output, openai_pred_class_paths_claude_output  = get_class_prediction_result_batched(claude_result_list, claude_item_list, max_workers=2)
# openai_pred_class_paths_claude_output = [item.split(" ")[1] if " " in item else item for item in openai_pred_class_paths_claude_output]

Processing Batches:   0%|          | 0/12 [00:00<?, ?it/s]


Prediction Summary:
Total items processed: 30
Number of batches: 6
Claude successful predictions: 25
OpenAI successful predictions: 30


In [39]:
correct_class_paths_openai_output, claude_pred_class_paths_openai_output, openai_pred_class_paths_openai_output  = get_class_prediction_result_batched(openai_result_list, openai_item_list, max_workers=2)
openai_pred_class_paths_openai_output = [item.split(" ")[1] if " " in item else item for item in openai_pred_class_paths_openai_output]

Processing Batches:   0%|          | 0/12 [00:00<?, ?it/s]


Prediction Summary:
Total items processed: 30
Number of batches: 6
Claude successful predictions: 10
OpenAI successful predictions: 30


#### Result from Feature Comparison

In [29]:
# print(f"Claude result using Claude for Feature Comparison: {count_matches(correct_class_paths_claude_output, claude_pred_class_paths_claude_output)}")
# print(f"Claude result using openai for Feature Comparison: {count_matches(correct_class_paths_claude_output, openai_pred_class_paths_claude_output)}")

Claude result using Claude for Feature Comparison: 3
Claude result using openai for Feature Comparison: 5


In [41]:
print(f"OpenAI result using openai for Feature Comparison: {count_matches(correct_class_paths_openai_output, openai_pred_class_paths_openai_output)}")
print(f"OpenAI result using Claude for Feature Comparison: {count_matches(correct_class_paths_openai_output, claude_pred_class_paths_openai_output)}")

OpenAI result using openai for Feature Comparison: 6
OpenAI result using Claude for Feature Comparison: 2
